In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from skimage.io import imread
from tifffile import imsave
from a72_utilities import *
from scipy.ndimage import shift
from skimage.registration import phase_cross_correlation
from skimage.transform import AffineTransform, warp

In [ ]:
xas_elements, xas_masks, xas_photon_energies = load_registered_xas()

In [ ]:
# cr_xas = imread('/Users/apple/Sync/Research/Alloy 72/20240614_Alloy_72_Native_Oxide/Cropped XAS images/XAS_30_um_2CA_2AN_570-584eV_0.2eV_step_Cr2p.tif')
# ni_xas = imread('/Users/apple/Sync/Research/Alloy 72/20240614_Alloy_72_Native_Oxide/Cropped XAS images/XAS_30um_2CA_2AN_848-865eV_0.2eV_step_Ni2p_aligned.tif')
# ti_xas = imread('/Users/apple/Sync/Research/Alloy 72/20240614_Alloy_72_Native_Oxide/Cropped XAS images/XAS_30um_2CA_2AN_452-470eV_0.2eV_step_Ti2p_aligned.tif')
# al_xas = imread('/Users/apple/Sync/Research/Alloy 72/20240614_Alloy_72_Native_Oxide/Cropped XAS images/XAS_30um_2CA_2AN_75-90eV_0.2eV_step_Al2p_aligned.tif')
# fe_xas = imread('/Users/apple/Sync/Research/Alloy 72/20240614_Alloy_72_Native_Oxide/Cropped XAS images/XAS_30um_2CA_2AN_703-718eV_0.2eV_step_Fe2p.tif')
# xas_elements = {"Cr":cr_xas,"Ni":ni_xas,"Ti":ti_xas,"Al":al_xas,"Fe":fe_xas}

# al_mask = imread('/Users/apple/Sync/Research/Alloy 72/Segmentation/Fiji segmentation/Al_frame_30_leveled_gaussian_blur.tif')
# cr_mask = imread('/Users/apple/Sync/Research/Alloy 72/Segmentation/Fiji segmentation/Cr_frame_30_leveled_gaussian_blur.tif')
# fe_mask = imread('/Users/apple/Sync/Research/Alloy 72/Segmentation/Fiji segmentation/Fe frame 15 gaussian blur only.tif')
# ni_mask = imread('/Users/apple/Sync/Research/Alloy 72/Segmentation/Fiji segmentation/Ni_frame_14_leveled_gaussian_blur.tif')
# ti_mask = imread('/Users/apple/Sync/Research/Alloy 72/Segmentation/Fiji segmentation/Ti frame 37 gaussian blur.tif')
# xas_masks = {"Cr":cr_mask,"Ni":ni_mask,"Ti":ti_mask,"Al":al_mask,"Fe":fe_mask}

# xas_photon_energies= {"Cr":np.linspace(570,584,len(cr_xas)), "Ni":np.linspace(848,865,len(ni_xas)), 
#                       "Ti":np.linspace(452,470,len(ti_xas)),"Al":np.linspace(75,90,len(al_xas)),
#                       "Fe":np.linspace(703,718,len(fe_xas))}
# print(len(cr_xas),len(ni_xas),len(ti_xas),len(al_xas),len(fe_xas))


In [ ]:
"""Use the phase cross-correlation to align the features each of the images"""

ref_img = xas_masks["Al"]
for i, (element,mask) in enumerate(xas_masks.items()):
    img_shift,err,phase = phase_cross_correlation(ref_img,mask)
    print(element)
    print(img_shift)
    shifted = shift(mask,img_shift)
    shifted_hyperspectral = np.zeros(xas_elements[element].shape)
    current_img = xas_elements[element]
    for frame in range(len(xas_elements[element])):
        shifted_hyperspectral[frame,:,:] = shift(current_img[frame,:,:],img_shift)
    
    #imsave('/Users/apple/Sync/Research/Alloy 72/Registration/'+element+'_shifted.tif',shifted_hyperspectral.astype(np.uint16))
    plt.imshow(shifted-ref_img)
    plt.show()

#imsave('/Users/apple/Sync/Research/Alloy 72/Registration/test.png', make_16bit())


Register the negative and positive polarity images from the TOF-SIMS measurements

In [ ]:
positive_polarity_masks = {"L21": imread('/Users/apple/Sync/Research/Alloy 72/A72 Tof SIMS/Masks/Positive polarity/l21 from Al+.tif'),"Ti":imread('/Users/apple/Sync/Research/Alloy 72/A72 Tof SIMS/Masks/Positive polarity/Ti-Ti ox pphase from TiO+.tif')}
negative_polarity_masks = {"L21": imread('/Users/apple/Sync/Research/Alloy 72/A72 Tof SIMS/Masks/Negative polarity/L21 from AlOx.tif'),"Ti":imread('/Users/apple/Sync/Research/Alloy 72/A72 Tof SIMS/Masks/Negative polarity/Ti rich from Ox.tif')}

In [ ]:
def find_shift_2d(ref_img,moving_img):
    img_shift,err,phase = phase_cross_correlation(ref_img,moving_img)
    shifted = shift(moving_img,img_shift)
    print(img_shift)
    plt.imshow(shifted-ref_img)
    plt.show()
    return shifted

In [ ]:
plt.imshow(find_shift_2d(positive_polarity_masks["L21"],negative_polarity_masks["L21"]))
plt.imshow(find_shift_2d(positive_polarity_masks["Ti"],negative_polarity_masks["Ti"]))

In [ ]:
def find_filename(directory, word, filetype=".txt"):
    matching_files = []
    
    # Walk through the directory and its subdirectories
    for root, dirs, files in os.walk(directory):
        for file in files:
            # Check if the file is a text file and contains the word in its name
            if file.endswith(filetype) and word.lower() in file.lower():
                matching_files.append((os.path.join(root, file)))
    
    return matching_files

In [ ]:
positive_polarity = find_filename('/Users/apple/Sync/Research/Alloy 72/A72 Tof SIMS/Raw data/Passive film component raw data','positive')
negative_polarity = find_filename('/Users/apple/Sync/Research/Alloy 72/A72 Tof SIMS/Raw data/Passive film component raw data','negative')

In [ ]:
for i, image_path in enumerate(negative_polarity):
    image = read_image_from_textfile(image_path)
    shifted = np.zeros(image.shape)
    shifted = shift(image,(14,-25))
    plt.imshow(shifted)
    plt.show()
    head, tail = os.path.split(image_path)
    #print(head+'/'+'registered_'+tail[:-4]+'.tif')
    imsave(os.path.join(head,'registered_'+tail[:-4]+'.tif'),shifted[25:,:-25].astype(np.uint16))


for i, image_path in enumerate(positive_polarity):
    image = read_image_from_textfile(image_path)
    plt.imshow(image)
    plt.show()
    head, tail = os.path.split(image_path)
    imsave(os.path.join(head,'registered_'+tail[:-4]+'.tif'),image[25:,:-25].astype(np.uint16))

Register the depth profiles from the Tof-SIMS measurements

In [ ]:
dir = '/Users/apple/Sync/Research/Alloy 72/A72 Tof SIMS/Raw data/Depth profile stacks'
depth_profiles = {}
for filename in os.listdir(dir):
    element = filename[:2]
    data = imread(os.path.join(dir, filename))
    depth_profiles[element] = data